In [108]:
source_url = "https://corporate.walmart.com/about/leadership"

In [109]:
!pip install requests-html lxml_html_clean

In [110]:
import pandas as pd
import numpy as np
from requests_html import AsyncHTMLSession
import json,re
import asyncio,itertools

import requests
from bs4 import BeautifulSoup


In [111]:

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
response = requests.get(source_url, headers=headers)

html_content = response.text

In [112]:
async def scrape_page(url):
    # Initialize session
    session = AsyncHTMLSession()
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:143.0) Gecko/20100101 Firefox/143.0',
               'Accept-Language': 'en-US,en;q=0.9',
                'Accept-Encoding': 'gzip, deflate, br, zstd',
                'Referer': 'https://www.google.com/',
                'Connection': 'keep-alive',
                'Upgrade-Insecure-Requests': '1',
                'Sec-Fetch-Dest': 'document',
                'Sec-Fetch-Mode': 'navigate',
                'Sec-Fetch-Site': 'cross-site',
                'Sec-Fetch-User': '?1',
              }


    # Load page and render JS
    response = await session.get(url,headers=headers)
    await response.html.arender(timeout=20, sleep=2)  # Render JS, wait 2s post-render

    if response.status_code!=200:
      print(f"The request failed with an error {response.status_code}. The Python scraping bot is being blocked by the website security")

    # Parse with BeautifulSoup
    soup = BeautifulSoup(response.html.html, 'html.parser')

    await session.close() # Close the session

    return soup

# Example usage


In [113]:
soup = BeautifulSoup(html_content, "lxml")

In [117]:
if(soup.find_all(["a","p","h1","h2","h3","h4","h5"])==[]):
  # Run the async function
  soup = await scrape_page(source_url)
  print(soup)


In [119]:
h1 = [str(x.get_text().strip()) for x in soup.find_all(['h1'])]
h1 = list(filter(None, h1))

h2 = [str(x.get_text().strip()) for x in soup.find_all(['h2'])]
h2 = list(filter(None, h2))

h3 = [str(x.get_text().strip()) for x in soup.find_all(['h3'])]
h3 = list(filter(None, h3))

h4 = [str(x.get_text().strip()) for x in soup.find_all(['h4'])]
h4 = list(filter(None, h4))

h5 = [str(x.get_text().strip()) for x in soup.find_all(['h5'])]
h5 = list(filter(None, h5))

h6 = [str(x.get_text().strip()) for x in soup.find_all(['h6'])]
h6 = list(filter(None, h6))

p = [str(x.get_text().strip()) for x in soup.find_all(['p'])]
p = list(filter(None, p))

a = [str(x.get_text().strip()) for x in soup.find_all(['a'])]
a = list(filter(None, a))

div = [str(x.get_text().strip()) for x in soup.find_all(['div'])]
div = list(filter(None, div))

span = [str(x.get_text().strip()) for x in soup.find_all(['span'])]
span = list(filter(None, span))

strong = [str(x.get_text().strip()) for x in soup.find_all(['strong'])]
strong = list(filter(None, strong))





In [ ]:
include_a_tag_flag=False


False

In [122]:
lists = [h1, h2, h3, h4, h5, h6, p, span]
lists_name=['h1','h2','h3','h4','h5','h6','p', 'span']
if(include_a_tag_flag):
  lists.append(a)
  lists_name.append('a')

# Keywords to search
keywords = ['officer', 'chief', 'ceo', 'partner', 'president','founder']
pattern = re.compile(r'\b(' + '|'.join(keywords) + r')\b', re.IGNORECASE)

# Count matches per list
results = []
for i, lst in enumerate(lists, 1):
    text = ' '.join(lst).lower()
    matches = pattern.findall(text)
    results.append({
        'list_index': i-1,
        'list_name': f'h{i}',
        'weightage': len(matches)
    })

# Find list with max weightage
max_result = max(results, key=lambda x: x['weightage'])
print(results)
designation_list = lists[max_result['list_index']]
print(designation_list)
print('Designation lists are at list: ',lists_name[max_result['list_index']],' /weightage :', max_result['weightage'])

[{'list_index': 0, 'list_name': 'h1', 'weightage': 0}, {'list_index': 1, 'list_name': 'h2', 'weightage': 0}, {'list_index': 2, 'list_name': 'h3', 'weightage': 0}, {'list_index': 3, 'list_name': 'h4', 'weightage': 0}, {'list_index': 4, 'list_name': 'h5', 'weightage': 0}, {'list_index': 5, 'list_name': 'h6', 'weightage': 0}, {'list_index': 6, 'list_name': 'h7', 'weightage': 127}, {'list_index': 7, 'list_name': 'h8', 'weightage': 0}]
['Powered by generative AI. Results may not fully be accurate. Use is governed by the\n                            \n                                Generative AI Guidelines\n                             and other applicable \n                                Walmart policies.', 'President and CEO, Walmart Inc.', 'Executive Vice President, Corporate Affairs, Walmart Inc.', 'Executive Vice President of Global Governance, Chief Legal Officer and Corporate Secretary, Walmart Inc.', 'Executive Vice President, AI Acceleration, Product and Design', 'President and CE

In [123]:
# Keywords to search
lists = [h1, h2, h3, h4, h5, h6]
lists_name=['h1','h2','h3','h4','h5','h6']
if(include_a_tag_flag):
  lists.append(a)
  lists_name.append('a')


name_pattern = re.compile(r'^[A-Za-z]+([- ][A-Za-z]+)*$', re.IGNORECASE)

name_pattern = re.compile(r"^(?:[A-Z][a-z]*(?:['.-][A-Z][a-z]*)*\s*){2,4}$")

#name_pattern = re.compile(r"^(?:[A-Z][\w\s.'-]*){2,4}$", re.UNICODE)

# Count matches per list
results = []
for i, lst in enumerate(lists, 1):
    matches = [s for s in lst if name_pattern.match(s)]
    results.append({
        'list_index': i-1,
        'list_name': f'h{i}',
        'weightage': len(matches)
    })
filtered_data = results
print(filtered_data)
############filter outlier value if the weightage is more########
if(not include_a_tag_flag):
  filtered_data=[]
  # Step 2: Calculate Q1, Q3, and IQR using numpy
  weightages = [d['weightage'] for d in results]
  q1, q3 = np.percentile(weightages, [25, 75])
  iqr = q3 - q1

  # Step 3: Define the outlier bounds
  lower_bound = q1 - 1.5 * iqr
  upper_bound = q3 + 1.5 * iqr

  # Step 4: Use a list comprehension to filter out dictionaries with outlier weightages
  filtered_data = [d for d in results if lower_bound <= d['weightage'] <= 150]
  print(filtered_data)
#############################

# Find list with max weightage
max_result = max(filtered_data, key=lambda x: x['weightage'])
print(max_result)

name_list = lists[max_result['list_index']]
#name_list = matches
print(name_list)
print('Name lists are at list: ',lists_name[max_result['list_index']],' /weightage :', max_result['weightage'])

[{'list_index': 0, 'list_name': 'h1', 'weightage': 0}, {'list_index': 1, 'list_name': 'h2', 'weightage': 2}, {'list_index': 2, 'list_name': 'h3', 'weightage': 0}, {'list_index': 3, 'list_name': 'h4', 'weightage': 0}, {'list_index': 4, 'list_name': 'h5', 'weightage': 52}, {'list_index': 5, 'list_name': 'h6', 'weightage': 0}]
[{'list_index': 0, 'list_name': 'h1', 'weightage': 0}, {'list_index': 1, 'list_name': 'h2', 'weightage': 2}, {'list_index': 2, 'list_name': 'h3', 'weightage': 0}, {'list_index': 3, 'list_name': 'h4', 'weightage': 0}, {'list_index': 4, 'list_name': 'h5', 'weightage': 52}, {'list_index': 5, 'list_name': 'h6', 'weightage': 0}]
{'list_index': 4, 'list_name': 'h5', 'weightage': 52}
['Doug McMillon', 'Dan Bartlett', 'Rachel Brand', 'Daniel Danker', 'John Furner', 'Suresh Kumar', 'Kathryn McLay', 'Donna Morris', 'Chris Nicholas', 'John David Rainey', 'Andrea Albright', 'Gordon Y. Allison', 'Bob Balfe', 'Julie Barber', 'Vinod Bidarkoppa', 'Dan Binder', 'Dan Bryant', 'David 

In [124]:
results

[{'list_index': 0, 'list_name': 'h1', 'weightage': 0},
 {'list_index': 1, 'list_name': 'h2', 'weightage': 2},
 {'list_index': 2, 'list_name': 'h3', 'weightage': 0},
 {'list_index': 3, 'list_name': 'h4', 'weightage': 0},
 {'list_index': 4, 'list_name': 'h5', 'weightage': 52},
 {'list_index': 5, 'list_name': 'h6', 'weightage': 0}]

In [125]:
name_list

['Doug McMillon',
 'Dan Bartlett',
 'Rachel Brand',
 'Daniel Danker',
 'John Furner',
 'Suresh Kumar',
 'Kathryn McLay',
 'Donna Morris',
 'Chris Nicholas',
 'John David Rainey',
 'Andrea Albright',
 'Gordon Y. Allison',
 'Bob Balfe',
 'Julie Barber',
 'Vinod Bidarkoppa',
 'Dan Binder',
 'Dan Bryant',
 'David M. Chojnowski',
 'Cedric Clark',
 'Isaac Cody',
 'Elisebeth Collins',
 'Megan Crozier',
 'Seth Dallaire',
 'Michael Ehret',
 'Chris Farrell',
 'Julie Gehrki',
 'Jerry R. Geisler III',
 'David Guggina',
 'Denise Incandela',
 'Sravana Karnati',
 'Silvia Azrai Kawas',
 'Kyle Kinnard',
 'Kerry Kotouc',
 'John Laney',
 'Larry Lundeen',
 'Claire MacIntyre',
 'Diana Marshall',
 'Kathleen McLaughlin',
 'Dwayne Milum',
 'Rob Montgomery',
 'Elizabeth Norton',
 'Allyson Park',
 'Chad Pekron',
 'Sanjay Radhakrishnan',
 'Lance de la Rosa',
 'Steve Schmitt',
 'Todd Sears',
 'Kieran Shanahan',
 'William Silcott',
 'Tim Simmons',
 'Jeff Spalding',
 'Hari Vasudev',
 'Tom Ward',
 'Latriece Watkins'

In [126]:
designation_list

['Powered by generative AI. Results may not fully be accurate. Use is governed by the\n                            \n                                Generative AI Guidelines\n                             and other applicable \n                                Walmart policies.',
 'President and CEO, Walmart Inc.',
 'Executive Vice President, Corporate Affairs, Walmart Inc.',
 'Executive Vice President of Global Governance, Chief Legal Officer and Corporate Secretary, Walmart Inc.',
 'Executive Vice President, AI Acceleration, Product and Design',
 'President and CEO, Walmart U.S.',
 'Executive Vice President, Global Chief Technology Officer and Chief Development Officer, Walmart Inc.',
 'President and CEO, Walmart International',
 'Executive Vice President and Chief People Officer, Walmart Inc.',
 "President and CEO, Sam's Club",
 'Executive Vice President and Chief Financial Officer, Walmart Inc.',
 'Executive Vice President and Chief Growth Officer, Walmart International; Operating Pa

In [127]:
remove_list_patterns = ["follow", "download","logo","skip","news",'subscribe','start','terms','help','reserved',\
                        'into the','exceptional','capabilities','model','committed','with','every'] # Patterns to match and remove

regex_pattern = "|".join(remove_list_patterns)


name_remove_list_patterns = ["follow", "download","logo","skip","news",'contact','diversity','&','connect',\
                             "press",'subscribe','start','digital','privacy','terms','help','wallet','account',\
                             'leaders','about','cloud','demo','pricing','login','free','promise','entertain',\
                             'enthusiasm','products','platform','resource','education','health','store','government','values',\
                             'analysis','source','software','microsoft','policy','studies','business','corporate','speed',\
                             'life','work','services','community','generative','overview','brand','data','engineer',\
                             'consult','read','compan','join','cpg','results','logged','vision','preference','relation',\
                             'analyst','security','tech','service','intelligen','individual','learn',\
                             'partner','ecosy','social','search','compan','equity','customer','director']
name_regex_pattern = "|".join(name_remove_list_patterns)

# 2. Filter the first list using a list comprehension
designation_list = [item for item in designation_list if not re.search(regex_pattern, item,re.IGNORECASE)]
name_list = [item for item in name_list if not re.search(name_regex_pattern, item,re.IGNORECASE)]
#removes 3 or more digit numbers in designation list
designation_list = [item for item in designation_list if not re.search(r'\d{3,}', item)]

name_list = [item for item in name_list if len(item.split()) <= 4 and len(item.split()) > 1]
name_list = [item for item in name_list if not re.search(r'\d{3,}', item)]

#name_list = [name for name in name_list if name_pattern.match(name)] #matches name regex







In [128]:
name_list

['Doug McMillon',
 'Dan Bartlett',
 'Daniel Danker',
 'John Furner',
 'Suresh Kumar',
 'Kathryn McLay',
 'Donna Morris',
 'Chris Nicholas',
 'John David Rainey',
 'Andrea Albright',
 'Gordon Y. Allison',
 'Bob Balfe',
 'Julie Barber',
 'Vinod Bidarkoppa',
 'Dan Binder',
 'Dan Bryant',
 'David M. Chojnowski',
 'Cedric Clark',
 'Isaac Cody',
 'Elisebeth Collins',
 'Megan Crozier',
 'Seth Dallaire',
 'Michael Ehret',
 'Chris Farrell',
 'Julie Gehrki',
 'Jerry R. Geisler III',
 'David Guggina',
 'Denise Incandela',
 'Sravana Karnati',
 'Silvia Azrai Kawas',
 'Kyle Kinnard',
 'Kerry Kotouc',
 'John Laney',
 'Larry Lundeen',
 'Claire MacIntyre',
 'Diana Marshall',
 'Kathleen McLaughlin',
 'Dwayne Milum',
 'Rob Montgomery',
 'Elizabeth Norton',
 'Allyson Park',
 'Chad Pekron',
 'Sanjay Radhakrishnan',
 'Lance de la Rosa',
 'Steve Schmitt',
 'Todd Sears',
 'Kieran Shanahan',
 'William Silcott',
 'Tim Simmons',
 'Jeff Spalding',
 'Hari Vasudev',
 'Tom Ward',
 'Latriece Watkins',
 'Judy Werthaus

In [129]:
designation_list

['Powered by generative AI. Results may not fully be accurate. Use is governed by the\n                            \n                                Generative AI Guidelines\n                             and other applicable \n                                Walmart policies.',
 'President and CEO, Walmart Inc.',
 'Executive Vice President, Corporate Affairs, Walmart Inc.',
 'Executive Vice President of Global Governance, Chief Legal Officer and Corporate Secretary, Walmart Inc.',
 'Executive Vice President, AI Acceleration, Product and Design',
 'President and CEO, Walmart U.S.',
 'Executive Vice President, Global Chief Technology Officer and Chief Development Officer, Walmart Inc.',
 'President and CEO, Walmart International',
 'Executive Vice President and Chief People Officer, Walmart Inc.',
 "President and CEO, Sam's Club",
 'Executive Vice President and Chief Financial Officer, Walmart Inc.',
 'Executive Vice President and Chief Growth Officer, Walmart International; Operating Pa

In [130]:

keywords = ['officer', 'chief', 'chairman','director','ceo','partner','president','founder','cfo','fellow','vp','senior','cto','lead']

designation_list = [title for title in designation_list if any(keyword in title.lower() for keyword in keywords)]

designation_list

['President and CEO, Walmart Inc.',
 'Executive Vice President, Corporate Affairs, Walmart Inc.',
 'Executive Vice President of Global Governance, Chief Legal Officer and Corporate Secretary, Walmart Inc.',
 'Executive Vice President, AI Acceleration, Product and Design',
 'President and CEO, Walmart U.S.',
 'Executive Vice President, Global Chief Technology Officer and Chief Development Officer, Walmart Inc.',
 'President and CEO, Walmart International',
 'Executive Vice President and Chief People Officer, Walmart Inc.',
 "President and CEO, Sam's Club",
 'Executive Vice President and Chief Financial Officer, Walmart Inc.',
 'Executive Vice President and Chief Growth Officer, Walmart International; Operating Partner, Massmart',
 'Senior Vice President, Office of the Corporate Secretary, and Chief Counsel for Finance and Corporate Governance',
 'Senior Vice President and Chief Counsel, Global Security, Aviation and Investigations, Walmart Inc.',
 'Executive Vice President and Chief Mer

In [131]:
unicode_chars_to_remove = ['\u200b', '\u00a0','\n','\u2028']


# Use a function to apply all replacements to a single string
def remove_multiple_chars(text, chars_to_remove):
    for char in chars_to_remove:
        text = text.replace(char, ' ')
    return text

name_list = [remove_multiple_chars(name, unicode_chars_to_remove) for name in name_list]
designation_list = [remove_multiple_chars(name, unicode_chars_to_remove) for name in designation_list]

In [132]:
print(len(name_list))
print(len(designation_list))

55
56


In [133]:
data = {'Name': name_list, 'Designation': designation_list}

data = list(itertools.zip_longest(name_list, designation_list))

# Create the DataFrame, specifying column names
df = pd.DataFrame(data,columns=['Name','Designation'])

df

,Name,Designation
0,Doug McMillon,"President and CEO, Walmart Inc."
1,Dan Bartlett,"Executive Vice President, Corporate Affairs, W..."
2,Daniel Danker,"Executive Vice President of Global Governance,..."
3,John Furner,"Executive Vice President, AI Acceleration, Pro..."
4,Suresh Kumar,"President and CEO, Walmart U.S."
5,Kathryn McLay,"Executive Vice President, Global Chief Technol..."
6,Donna Morris,"President and CEO, Walmart International"
7,Chris Nicholas,Executive Vice President and Chief People Offi...
8,John David Rainey,"President and CEO, Sam's Club"
9,Andrea Albright,Executive Vice President and Chief Financial O...


In [134]:
df[['First Name', 'Last Name']] = df['Name'].str.split(n=1, expand=True)

# Convert the new columns to lowercase
df['First Name'] = df['First Name'].str.lower()
df['Last Name'] = df['Last Name'].str.lower()

df

,Name,Designation,First Name,Last Name
0,Doug McMillon,"President and CEO, Walmart Inc.",doug,mcmillon
1,Dan Bartlett,"Executive Vice President, Corporate Affairs, W...",dan,bartlett
2,Daniel Danker,"Executive Vice President of Global Governance,...",daniel,danker
3,John Furner,"Executive Vice President, AI Acceleration, Pro...",john,furner
4,Suresh Kumar,"President and CEO, Walmart U.S.",suresh,kumar
5,Kathryn McLay,"Executive Vice President, Global Chief Technol...",kathryn,mclay
6,Donna Morris,"President and CEO, Walmart International",donna,morris
7,Chris Nicholas,Executive Vice President and Chief People Offi...,chris,nicholas
8,John David Rainey,"President and CEO, Sam's Club",john,david rainey
9,Andrea Albright,Executive Vice President and Chief Financial O...,andrea,albright


In [135]:
img_tags = soup.find_all('img')
src_list = [img['src'] for img in img_tags if 'src' in img.attrs]                   #appending some image list present in src tag
data_src_list = [img['data-src'] for img in img_tags if 'data-src' in img.attrs]    #appending some image list present in data-src tag

src_list.extend(data_src_list)

src_list





['/etc.clientlibs/adp/clientlibs/clientlib-site/resources/images/Corp-Hamburger.svg',
 '/content/dam/corporate/site-images/WMT-Spark-New-SparkYellow-RGB.svg',
 '/etc.clientlibs/adp/clientlibs/clientlib-site/resources/images/language-global.svg',
 '/etc.clientlibs/adp/clientlibs/clientlib-site/resources/images/corporate-icons/giftBag.svg',
 '/etc.clientlibs/adp/clientlibs/clientlib-site/resources/images/language-global-black.svg',
 '/content/dam/corporate/site-images/WMT-Wordmark-New-Small-Black_RGB.svg',
 '/etc.clientlibs/adp/clientlibs/clientlib-site/resources/images/yourprivacychoice.png',
 '/content/dam/corporate/site-images/footer-logos/Walmart-US-Text-Logo-Black.png',
 '/content/dam/corporate/site-images/footer-logos/Sams-Club-Text-Logo-Black.png',
 '/content/dam/corporate/site-images/footer-logos/Walmart-International-Text-Logo-Black.png',
 '/content/dam/corporate/site-images/icons/X-glyph-black-sm.png',
 '/etc.clientlibs/adp/clientlibs/clientlib-site/resources/images/yourprivacy

In [136]:
df['img url']=None
for index, row in df.iterrows():
    first_name = row['First Name']
    last_name = row['Last Name']

    # Check if the first name is present in any of the URLs
    for url in src_list:
        # Convert both name and URL to lowercase for case-insensitive matching
        if first_name is not None and first_name in str(url).lower():
            # If a match is found, assign the full URL to the new column
            df.at[index, 'img url'] = url
            # Break the inner loop once a match is found
            break
    for url in src_list:
        # Convert both name and URL to lowercase for case-insensitive matching
        if last_name is not None and last_name in str(url).lower():
            # If a match is found, assign the full URL to the new column
            df.at[index, 'img url'] = url
            # Break the inner loop once a match is found
            break

In [137]:
full_url_flag = False
if(((str(df['img url'][1])[:8]) or (str(df['img url'][0])[:8]) or (str(df['img url'][-1])[:8])) == 'https://'):
  full_url_flag = True

In [138]:
stripped_url = source_url[:source_url.find('.com') + len('.com')]

if(not full_url_flag):
  df['img url']=stripped_url+df['img url']

df.loc[df['img url'].duplicated(keep='first'), 'img url'] = None

df.drop(columns=['First Name','Last Name'],inplace=True)
df

,Name,Designation,img url
0,Doug McMillon,"President and CEO, Walmart Inc.",https://corporate.walmart.com/content/dam/corp...
1,Dan Bartlett,"Executive Vice President, Corporate Affairs, W...",https://corporate.walmart.com/content/dam/corp...
2,Daniel Danker,"Executive Vice President of Global Governance,...",https://corporate.walmart.com/content/dam/corp...
3,John Furner,"Executive Vice President, AI Acceleration, Pro...",https://corporate.walmart.com/content/dam/corp...
4,Suresh Kumar,"President and CEO, Walmart U.S.",https://corporate.walmart.com/content/dam/corp...
5,Kathryn McLay,"Executive Vice President, Global Chief Technol...",https://corporate.walmart.com/content/dam/corp...
6,Donna Morris,"President and CEO, Walmart International",https://corporate.walmart.com/content/dam/corp...
7,Chris Nicholas,Executive Vice President and Chief People Offi...,https://corporate.walmart.com/content/dam/corp...
8,John David Rainey,"President and CEO, Sam's Club",None
9,Andrea Albright,Executive Vice President and Chief Financial O...,NaN


In [139]:
df['Name']=df['Name'].fillna('')
df['Designation']=df['Designation'].fillna('')

In [140]:
df.to_json(orient='records',indent=3)


'[\n   {\n      "Name":"Doug McMillon",\n      "Designation":"President and CEO, Walmart Inc.",\n      "img url":"https:\\/\\/corporate.walmart.com\\/content\\/dam\\/corporate\\/images\\/about\\/leadership\\/executive-leadership\\/dougmcmillon-img-236x170-v2.jpg"\n   },\n   {\n      "Name":"Dan Bartlett",\n      "Designation":"Executive Vice President, Corporate Affairs, Walmart Inc.",\n      "img url":"https:\\/\\/corporate.walmart.com\\/content\\/dam\\/corporate\\/images\\/about\\/leadership\\/executive-leadership\\/danbartlett-img-236x170-v2.jpg"\n   },\n   {\n      "Name":"Daniel Danker",\n      "Designation":"Executive Vice President of Global Governance, Chief Legal Officer and Corporate Secretary, Walmart Inc.",\n      "img url":"https:\\/\\/corporate.walmart.com\\/content\\/dam\\/corporate\\/images\\/about\\/leadership\\/executive-leadership\\/daniel-danker.jpg"\n   },\n   {\n      "Name":"John Furner",\n      "Designation":"Executive Vice President, AI Acceleration, Product an

In [141]:
df.to_dict(orient='records')

[{'Name': 'Doug McMillon',
  'Designation': 'President and CEO, Walmart Inc.',
  'img url': 'https://corporate.walmart.com/content/dam/corporate/images/about/leadership/executive-leadership/dougmcmillon-img-236x170-v2.jpg'},
 {'Name': 'Dan Bartlett',
  'Designation': 'Executive Vice President, Corporate Affairs, Walmart Inc.',
  'img url': 'https://corporate.walmart.com/content/dam/corporate/images/about/leadership/executive-leadership/danbartlett-img-236x170-v2.jpg'},
 {'Name': 'Daniel Danker',
  'Designation': 'Executive Vice President of Global Governance, Chief Legal Officer and Corporate Secretary, Walmart Inc.',
  'img url': 'https://corporate.walmart.com/content/dam/corporate/images/about/leadership/executive-leadership/daniel-danker.jpg'},
 {'Name': 'John Furner',
  'Designation': 'Executive Vice President, AI Acceleration, Product and Design',
  'img url': 'https://corporate.walmart.com/content/dam/corporate/images/about/leadership/executive-leadership/johnfurner-img-236x170-v

In [142]:
data = df.to_dict(orient='records')

In [143]:
# import requests
# from bs4 import BeautifulSoup
# import re

# def scrape_user_profile(url):
#     headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
#     response = requests.get(url, headers=headers)
#     if response.status_code != 200:
#         return {"error": f"Failed to fetch page: {response.status_code}"}

#     soup = BeautifulSoup(response.text, 'html.parser')

#     profile_data = {
#         "name": None,
#         "title": None,
#         "email": None,
#         "image": None,
#         "bio": None,
#         "social_links": [],
#         "professional_background": None,
#         "contact_info": None,
#         "other_details": None
#     }

#     # Name: Often in <h1> or class="author-name"
#     name_tag = soup.find('h1', class_='blog-title') or soup.find('h1')
#     if name_tag:
#         profile_data["name"] = name_tag.text.strip()
#     else:
#         # Fallback to meta or other tags
#         meta_name = soup.find('meta', attrs={'name': 'author'})
#         if meta_name:
#             profile_data["name"] = meta_name['content']

#     # Title: Look for <h2> or class="title"
#     title_tag = soup.find('h2', string=re.compile('author|title', re.I))
#     if title_tag:
#         profile_data["title"] = title_tag.text.strip()

#     # Email: Search for mailto links or regex in text
#     email_links = soup.find_all('a', href=re.compile(r'^mailto:'))
#     if email_links:
#         profile_data["email"] = email_links[0]['href'].replace('mailto:', '')
#     else:
#         # Regex fallback in text
#         email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
#         emails = re.findall(email_pattern, response.text)
#         if emails:
#             profile_data["email"] = emails[0]

#     # Image: Find profile images (class or alt containing "author" or "profile")
#     image_tag = soup.find('img', alt=re.compile('author|profile|headshot', re.I)) or soup.find('img', class_='author-image')
#     if image_tag:
#         profile_data["image"] = image_tag['src'] if image_tag['src'].startswith('http') else url + image_tag['src']

#     # Bio: Find <div class="bio"> or <p> in about section
#     bio_section = soup.find('div', class_='bio') or soup.find('section', id='about') or soup.find('p', string=re.compile('about me|bio', re.I))
#     if bio_section:
#         profile_data["bio"] = ' '.join(bio_section.text.strip().split())

#     # Social Links: Find <a> with href for common social platforms
#     social_patterns = ['twitter.com', 'linkedin.com', 'facebook.com', 'instagram.com', 'github.com']
#     for link in soup.find_all('a', href=True):
#         for pattern in social_patterns:
#             if pattern in link['href']:
#                 profile_data["social_links"].append(link['href'])

#     # Professional Background: Look for <section> or <div> with "background" or "experience"
#     background_section = soup.find('section', id='background') or soup.find('div', class_='professional-background') or soup.find('p', string=re.compile('background|experience', re.I))
#     if background_section:
#         profile_data["professional_background"] = ' '.join(background_section.text.strip().split())

#     # Contact Info: Look for contact sections or phone/emails
#     contact_section = soup.find('section', id='contact') or soup.find('div', class_='contact')
#     if contact_section:
#         profile_data["contact_info"] = ' '.join(contact_section.text.strip().split())

#     # Other Details: Catch-all for footer or meta
#     footer = soup.find('footer')
#     if footer:
#         profile_data["other_details"] = ' '.join(footer.text.strip().split())

#     return profile_data

# # Example usage
# url = "https://medium.com/the-ai-forum/create-a-sql-agent-using-crewai-and-groq-005895ba31b3"
# data = scrape_user_profile(url)
# print(data)

In [144]:
# from requests_html import AsyncHTMLSession
# from bs4 import BeautifulSoup
# import json
# import asyncio

# async def scrape_page(url):
#     # Initialize session
#     session = AsyncHTMLSession()
#     headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
#                'Accept-Language': 'en-US,en;q=0.9',
#                 'Accept-Encoding': 'gzip, deflate, br',
#                 'Referer': 'https://www.google.com/'
#               }


#     # Load page and render JS
#     response = await session.get(url,headers=headers)
#     await response.html.arender(timeout=20, sleep=2)  # Render JS, wait 2s post-render
#     print(response.status_code)

#     # Parse with BeautifulSoup
#     soup = BeautifulSoup(response.html.html, 'html.parser')

#     await session.close() # Close the session

#     return soup

# # Example usage
# url = f"https://mathco.com/our-company/#meet-our-leaders"

# # Run the async function
# soup = await scrape_page(url)
# print(soup)